In [ ]:
# ready for use 053018
# script for preparing eye tracking data to be used for a trial by trial analysis
# Analysis Step 2

In [16]:
import numpy as np
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

In [97]:
import pickle
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [98]:
behav_df = pd.read_excel('C:\\Users\\Christine\\Documents\\Dropbox\\dissertation\\MRI_task\\MRI_dissertation_subsALL_Dec11_2017.xlsx', sheet_name='MRI_dissertation_subsALL_121117')

In [99]:
# subset to get subject's data for n block
behav_info_dict = {}
prompt_info_dict = {}
sub_list = [32] #[3, 16, 18]
block_list = [1, 2, 3, 4, 5]
for sub in sub_list:
    s_text = "Sub%d" % sub
    behav_info_dict[s_text] = []
    sub_behav = behav_df.loc[behav_df['subject_ID'] == s_text]
    for block in block_list:
        b_text = "R%d" % block
        sub_behav_block = sub_behav.loc[sub_behav['block'] == b_text]
        key_name = "subject%d_block%d" % (sub, block)
        behav_info_dict[key_name] = sub_behav_block
        #getting columns with prompt and RTT information
        prompt_info = sub_behav_block.loc[:,['subject_ID', 'tap_period_duration', 'use_in_analysis', 'timing_4', 'promptRESP_RFA', 
                                            'env_promptRESP_RFA', 'fm_promptRESP_RFA', 'block', 'RTT']]
        prompt_info = prompt_info.sort_values(by=['timing_4'])
        prompt_key_name = "prompt_info_sub%d_block%d" % (sub, block)
        prompt_info_dict[prompt_key_name] = prompt_info
        
save_obj(behav_info_dict, 'behav_info_dict' )
save_obj(prompt_info_dict, 'prompt_info_dict' )

In [100]:
block_list = [1, 2, 3, 4, 5]
for sub in sub_list:
    eye_dict={}
    for block in block_list:
        eye_data_file = 'block%d_sub%d.csv' % (block, sub)
        eye_data = pd.read_csv(eye_data_file)
        key_name = "subject%d_block%d" % (sub, block)
        block_data = behav_info_dict.get(key_name)
        block_data = block_data.sort_values(by=['timing_4'])
        # select the tap column, which has already been sorted in order of presentation
        taps = block_data[['tap_period_duration']]
        taps['tap_totals'] = 12
        taps.loc[taps['tap_period_duration'] == 20, 'tap_totals'] = 15
        taps.loc[taps['tap_period_duration'] == 24, 'tap_totals'] = 18
        taps.loc[taps['tap_period_duration'] == 28, 'tap_totals'] = 22
        taps.loc[taps['tap_period_duration'] == 32, 'tap_totals'] = 25
        taps.loc[taps['tap_period_duration'] == 36, 'tap_totals'] = 28
        
        # running total of tap instances to match with Trial ID
        
        # starting point of Trial ID and sample message text for each block
        if block == 1:
            add_this = 0
            sm_string = "TRIALID 1"
        elif block == 2:
            add_this = 250
            sm_string = "TRIALID 251"
        elif block == 3:
            add_this = 500
            sm_string = "TRIALID 501"
        elif block == 4:
            add_this = 750
            sm_string = "TRIALID 751"
        elif block == 5:
            add_this = 1000
            sm_string = "TRIALID 1001"
                
        b = []
        btemp = taps.loc[taps.index[0], 'tap_totals'] + add_this
        for a in range(1, len(taps)):
            btemp = (btemp + taps.loc[taps.index[a], 'tap_totals'])
            btemp1 = btemp
            b.append(btemp1)
        
        
        first_row = taps.loc[taps.index[0], 'tap_totals'] + add_this
        b.insert(0, first_row)
        df=pd.DataFrame(b,columns=['trial_ID'])
        # resetting row indexes and concatenating data
        taps = taps.reset_index()
        df = df.reset_index()
        all_tap_data = pd.concat([taps, df], axis=1)
        
        # getting first trial baseline, which always precedes TRIALID 1
        first_base_z = list(np.where(eye_data['SAMPLE_MESSAGE'] == sm_string)[0])
        first_base_a = first_base_z[0] - 1000
        first_base_data = eye_data.loc[first_base_a:first_base_z[0]]
        
        all_tap_data['trial_start'] = all_tap_data['trial_ID'] - 4
        all_tap_filename = "trialIDS_block%d_sub%d.csv" % (block, sub)
        all_tap_data.to_csv(all_tap_filename)
        
# getting indexes for the trial IDs and where each trial starts
        
        trial_dict={}
        for t in range(0,len(all_tap_data)):
            trialStart_value = all_tap_data.loc[all_tap_data.index[t], 'trial_start']
            trialStart_string = "TRIALID %d" % (trialStart_value)
            trialStart_index = list(np.where(eye_data['SAMPLE_MESSAGE'] == trialStart_string)[0])
            trialID_value = all_tap_data.loc[all_tap_data.index[t], 'trial_ID']
            trialID_string = "TRIALID %d" % (trialID_value)
            trialID_index = list(np.where(eye_data['SAMPLE_MESSAGE'] == trialID_string)[0])
            trialID_index2 = trialID_index[0] + 1300
            # use indexes to subset trial data
            trial_data = eye_data.loc[trialStart_index[0]:trialID_index2]
            trial_data_dict_key = "trial%d_sub%d_block%d" % (trialID_value, sub, block)
            trial_dict[trial_data_dict_key] = trial_data
        
        # subsetting these two columns in preparation for baseline data
        trialID_column = all_tap_data['trial_ID']
        tap_totals_column = all_tap_data['tap_totals']
        
        # for all trials except Trial 1, get the last 1000 ms of baseline from beginning of tap period; store to dict
        baseline_indexes = trialID_column - (tap_totals_column - 1)
        baseline_dict={}
        for base in range(1,len(baseline_indexes)):
            baseline_trialID = baseline_indexes[base]
            baseline_trialID_string = "TRIALID %d" % (baseline_trialID)
            baseline_trialID_index_z = list(np.where(eye_data['SAMPLE_MESSAGE'] == baseline_trialID_string)[0])
            baseline_trialID_index_a = baseline_trialID_index_z[0] - 1000
            trial_base_data = eye_data.loc[baseline_trialID_index_a:baseline_trialID_index_z[0]]
            base_dict_key = "baseline_trial%s_sub%d_block%d" % ((trialID_column[base]), sub, block)
            baseline_dict[base_dict_key] = trial_base_data
            
        #create dictionary for each block for each subject
        sub_dict={}
        sub_dict_key1 = "all_tap_data"
        sub_dict_key2 = "all_trial_data"
        sub_dict_key3 = "first_baseline_data"
        sub_dict_key4 = "trials_baseline_data"
        sub_dict[sub_dict_key1] = all_tap_data
        sub_dict[sub_dict_key2] = trial_dict
        sub_dict[sub_dict_key3] = first_base_data
        sub_dict[sub_dict_key4] = baseline_dict
        
        eye_dict_key = "eye_data_sub%d_block%d" % (sub, block)
        eye_dict[eye_dict_key] = sub_dict
       
    name = 'eye_data_prepared_sub%d' % sub
    obj = eye_dict
    save_obj(obj, name )

C:\Users\Christine\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Users\Christine\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\Christine\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.